In [3]:
import os
import json
import sys

f = open('./data/meta_Movies_and_TV.json', 'r', encoding='utf-8')
data = []
i = 0
for l in f.readlines():
  dic = json.loads(l)
  data.append(dic)

In [4]:
def correct_spaces_and_symbols(input_string):
    cleaned_string = ' '.join(input_string.split())
    cleaned_string = cleaned_string.replace('. ', '.')
    cleaned_string = cleaned_string.replace(':', ': ')
    
    cleaned_string = ' '.join(cleaned_string.split())

    return cleaned_string

In [5]:
ID_title_dict = {"NULL": 0}
title_ID_dict = {"NULL": 0}
dataset = {}
for sample in data:
    sample["title"] = sample["title"].strip()
    sample["title"] = sample["title"].replace("&amp;", "&")
    sample["title"] = correct_spaces_and_symbols(sample["title"])
    ID_title_dict[sample["asin"]] = sample["title"]
    title_ID_dict[sample["title"]] = sample["asin"]
        
    if sample["asin"] == "" or sample["title"] == "":
        continue
    if len(sample['category']) <= 1:
        continue

    sample["also_view"].extend(sample["also_buy"])
    history_list = sample["also_view"][-50:]
    if len(history_list) < 5:
        continue
    dataset[sample["asin"]] = {"history": history_list, "category": sample['category']}

In [8]:
for id in dataset.keys():
    clicks = []
    for click_id in dataset[id]["history"]:
        if click_id in dataset:
            clicks.append(click_id)
    dataset[id]["history"] = clicks

In [9]:
import random
sample_num = 500
select_sampleID = random.sample(dataset.keys(), sample_num)
print(select_sampleID)

['B0142YXYPW', 'B003RACH88', 'B0069B0F72', 'B001CIOCMQ', 'B001QDAFIM', 'B008H6GHG0', 'B00L9S1VZK', 'B00008YLUN', 'B000VUQ4FE', '1562196308', 'B000WM84UO', '6302409829', 'B00XV97FEQ', 'B00WLL7X6K', 'B011MUA5J2', 'B007LNBSLE', 'B005DL6PSC', 'B01995ZFYM', 'B00PTUB45Q', '7885727513', '1611800277', 'B000FJTV9C', 'B00BKQ12F0', 'B00HL2HGEI', 'B00SEAAZ7G', 'B002AMVE9W', 'B009FFHWRU', 'B000640XOS', 'B00UL90D4I', 'B001MEJY96', '6305293651', 'B004HV1NMM', 'B002Q1KN6M', 'B00004WMMY', 'B0006J28E6', 'B000HT3862', 'B000HD1MUM', 'B00005UF7N', 'B00C9VZ73S', '5553799260', 'B001502LNI', 'B00J2FIQBY', 'B0056031N2', 'B001N6FPQY', 'B00VX21VWK', 'B0015RB6MI', 'B000OONQAI', 'B00440OFZS', 'B004FKKU92', 'B001L4L8UA', 'B0009Y276C', 'B00IZE00MQ', 'B004LGGXI2', 'B00PLYU8JS', 'B00005BCP6', 'B000PUAIMU', 'B001AVQJH6', 'B000DN5VN0', 'B004K9WWAI', 'B00000ICEY', 'B00JPSYPCM', '6301016416', 'B00QJAZ514', 'B0000AGQ68', 'B001D7T45Q', 'B000067IZL', 'B00IIQY08Q', 'B000RXZL0E', 'B00I2YEHAA', 'B00BKX7PJ0', 'B001FPKJNM', 'B00C

C:\Users\sdsc\AppData\Local\Temp\ipykernel_19188\3964593522.py:3: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  select_sampleID = random.sample(dataset.keys(), sample_num)


In [10]:
from collections import defaultdict
ID_tree = {}

def update_tree(id, cate_words, tree_dict):
    for i in range(0, len(cate_words)-1):
        if cate_words[i] not in tree_dict:
            tree_dict[cate_words[i]] = {"next": set(), "IDs": set()}
        tree_dict[cate_words[i]]["next"].add(cate_words[i+1])

    if cate_words[-1] not in tree_dict:
        tree_dict[cate_words[-1]] = {"next": set(), "IDs": set()}
    tree_dict[cate_words[-1]]["IDs"].add(str(id))

for id in dataset.keys():
    if id in select_sampleID:
        if len(dataset[id]["category"]) == 0 or dataset[id]["category"][-1] == "</span></span></span>":
            continue
        update_tree(id, dataset[id]["category"], ID_tree)

In [13]:
def subset_size(cate_list, tree_dict):
    if cate_list[-1] not in tree_dict:
        return 0
    return len(tree_dict[cate_list[-1]]["IDs"])

max_size = 50
for id in dataset.keys():
    if len(dataset[id]["category"]) == 0 or dataset[id]["category"][-1] == "</span></span></span>":
        continue
    if subset_size(dataset[id]["category"], ID_tree) < max_size:
        update_tree(id, dataset[id]["category"], ID_tree)

In [14]:
def build_prompt_stage_1(history_IDlist, ID_title_dict):
    # STAGE 1
    prompt = "Previously, the user has watched movies and tv as follows:\n"
    for id in history_IDlist:
        if id in ID_title_dict:
            prompt += ID_title_dict[id] + "\n"
    prompt += "Summarize the interested topics, from the most important to the least important."
    return prompt

def get_subset_list(items_ID, ID_title_dict):
    l = []
    for itemid in items_ID:
        if itemid in ID_title_dict:
            l.append(ID_title_dict[itemid])

    return l

def prompt_recall_item(items_ID, ID_title_dict, recall_num, cate_name):
    # STAGE 2
    prompt = "Select the top {} movies and TV shows from the following candidate set about {} ".format(recall_num, cate_name)
    prompt += "to recommend to the user, ranked from the most important to the least important, without offering any explanations. "
    prompt += "Please do not change the format of the movie or TV show titles during the output process."
    prompt += "Here is the provided list:\n"
    index = 1
    for itemid in items_ID:
        if itemid in ID_title_dict:
            prompt += "{}. {}\n".format(index, ID_title_dict[itemid])
            index += 1
    
    return prompt

def prompt_select_subcate(subcate_list, k, cate):
    # STAGE 2
    prompt = "Rank the top {} subcategories about {} from provided list based on the user's interest without any explanations:\n".format(k, cate)
    
    index = 1
    for subcate in subcate_list:
        prompt += "{}. {}\n".format(index, subcate)
        index += 1
    
    prompt += "Do not output any subcategory that do not appear in the list above.\n"
    
    return prompt

def build_prompt_rerank(cate_title_list):
    # STAGE 3
    prompt_rerank = "Rank these pre-selected movies and tv based on user interests. "
    prompt_rerank += "Be aware of ranking diversity and do not change the format of the title.\n"
    index = 1
    for subcate, itemslist in cate_title_list:
        for item in itemslist:
          prompt_rerank += "{}. {}\n".format(index, item)
          index += 1
    
    return prompt_rerank

In [16]:
def get_answer_list(message):
    lines = message.split('\n')
    cate_list = []
    for l in lines:
        if len(l) == 0:
            continue
        if l[0].isdigit():
            cate_name = l[l.find('.')+1:].strip()
            cate_list.append(cate_name)

    return cate_list

In [18]:
  # ChatGPT API

!pip install openai==0.27
import random
import time
import openai
import os

key_list = ["API_KEYS"]

class Conversation:
    def __init__(self, prompt, num_of_round):
        self.prompt = prompt
        self.num_of_round = num_of_round
        self.messages = []
        self.messages.append({"role": "system", "content": self.prompt})
        self.key_index = 0
        self.key_list = key_list
        self.key_list_len = len(self.key_list)

    def get_api_key(self):
        api_key = self.key_list[self.key_index]
        self.key_index += 1
        self.key_index %= self.key_list_len
        return api_key

    def initialize_openai_client(self, api_key):
        openai.api_key = api_key

    def ask(self, question):
        try:
            self.messages.append({"role": "user", "content": question})
            response = openai.ChatCompletion.create(
              model="gpt-3.5-turbo",
              messages=self.messages,
              temperature=0.5,
              max_tokens=1024,
              top_p=1,
            )
        except Exception as e:
            return e

        message = response["choices"][0]["message"]["content"]
        self.messages.append({"role": "assistant", "content": message})

        return message

    def remain_fisrtKRound(self, k):
        self.messages = self.messages[:1+2*k]

    def remain_firstKReponse(self, k):
        temp_list = []
        for msg in self.messages:
            if msg["role"] == "assistant":
                temp_list.append(msg)

        if len(temp_list) > k:
           temp_list = temp_list[:k]

        self.messages = temp_list

    def call_chatgpt(self, question, round, max_attempts=None):
        if max_attempts == None:
            max_attempts = self.key_list_len
        for _ in range(max_attempts):
            api_key = self.get_api_key()
            self.initialize_openai_client(api_key)
            message = None
            try:
                self.messages.append({"role": "user", "content": question})
                response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=self.messages,
                    temperature=0.5,
                    max_tokens=1024,
                    top_p=1
                )
                message = response["choices"][0]["message"]["content"]
                self.messages.append({"role": "assistant", "content": message})
        
                if response.choices:
                    return message
        
            except (openai.error.RateLimitError, openai.error.AuthenticationError) as e:
                # print(f"API Error: {e}")
                self.remain_fisrtKRound[round-1]
                time.sleep(20)
                continue
        
            except Exception as e:
                print(f"Unknown Error: {e}")
                continue
        return None

In [ ]:
import time
chatgpt = Conversation("", 30)
recall_subset = 4
steps = 50
Log_file = "AmazonLog.txt"

with open(Log_file, "w") as f:
    for user_id in select_sampleID:
        print("User ID:", user_id, file = f)
        if len(dataset[user_id]["history"]) == 0:
            print("User history too short: ", user_id)
            continue
        # STAGE 1
        prompt1 = build_prompt_stage_1(dataset[user_id]["history"], ID_title_dict)
        print("USER:", file = f)
        print(prompt1, file = f)
        message1 = chatgpt.call_chatgpt(prompt1, 1)
        print("ChatGPT:", file = f)
        print(message1, file = f)
    
        # STAGE 2
        recall_all = 20
        stack = ["Movies & TV"] # Root node
        recall_list = []
        cate_title_list = []

        leaf_i = 0
        
        import sys
        cate_count = defaultdict(int)
        while len(recall_list) < recall_all:
            time.sleep(20)
            if len(stack) == 0:
                break
            cate = stack[-1]
            stack.pop()
            print(cate)
            cate_count[cate] += 1
            if cate_count[cate] > 5:
                sys.exit(-1)
            if len(ID_tree[cate]["next"]) == 0:
                subset_size = len(ID_tree[cate]["IDs"])
                recall_num = min(subset_size, recall_subset)
                if recall_num == 0:
                    continue
            
                chatgpt.remain_fisrtKRound(1)
                item_title_list = get_subset_list(list(ID_tree[cate]["IDs"]), ID_title_dict)
                
                prompt2 = prompt_recall_item(list(ID_tree[cate]["IDs"]), ID_title_dict, recall_num, cate)
                if len(item_title_list) == 1:
                    message2 = "1. {}".format(item_title_list[0])
                else:
                    message2 = chatgpt.call_chatgpt(prompt2, 2)
                subset_list = get_answer_list(message2)
                if len(subset_list) == 0 or subset_list[-1] not in item_title_list:
                    stack.append(cate)
                    continue
                leaf_i += 1  
                print("USER:", file = f)
                print(prompt2, file = f)
                print("ChatGPT:", file = f)
                print(message2, file = f)
                recall_list.extend(subset_list)
                cate_title_list.append((cate, subset_list))
            else:
                subset_size = len(ID_tree[cate]["next"])
                recall_num = min(subset_size, recall_subset)
                if recall_num == 0:
                    continue
        
                prompt2 = prompt_select_subcate(list(ID_tree[cate]["next"]), recall_num, cate)
                chatgpt.remain_fisrtKRound(1)
                message2 = chatgpt.call_chatgpt(prompt2, 2)
                subset_list = get_answer_list(message2)
                if len(subset_list) == 0:
                    stack.append(cate)
                    continue
                replay = False
                for subcate in subset_list:
                    if subcate not in ID_tree.keys():
                        replay = True
                if replay:
                    stack.append(cate)
                    continue
        
        
                print("USER:", file = f)
                print(prompt2, file = f)
                print("ChatGPT:", file = f)
                print(message2, file = f)
                
                for subcate in subset_list[::-1]:
                    stack.append(subcate)
        
        # STAGE 3
        # Re-rank
        prompt3 = build_prompt_rerank(cate_title_list)
        print("USER:", file = f)
        print(prompt3, file = f)
        chatgpt.remain_fisrtKRound(1)
        message3 = chatgpt.call_chatgpt(prompt3, 2)
        print("ChatGPT:", file = f)
        print(message3, file = f)
        
        # Evaluation
        rerank_list = get_answer_list(message3)
        recall_success = 0
        for title in rerank_list:
            if title in title_ID_dict and title_ID_dict[title] == user_id:
                recall_success = 1
        print("Recall_rate: {}\n".format(recall_success), file = f)